# Imperative Inference (no `TensorFlow`!)

In [ ]:
import os
import sys
import time
import pickle

#import autograd.numpy as np
import numpy as np
import pandas as pd
#import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=3, linewidth=500, edgeitems=15, suppress=True)

import seaborn as sns
sns.set_palette('bright')


import matplotlib as mpl
import matplotlib.style as mplstyle

label_size = 9 
mpl.rcParams['xtick.labelsize'] = label_size 
mpl.rcParams['ytick.labelsize'] = label_size 

mplstyle.use('fast')

#from matplotlib import rc
#rc('text', usetex=False)

import utils.file_io as io

In [ ]:
import time
from plot_script import get_matching_log_dirs
t0 = time.time()
root_dir = os.path.abspath('../../gauge_logs/')
dates = ['2019_12_15',
         '2019_12_16',
         '2019_12_17',
         '2019_12_18',
         '2019_12_19',
         '2019_12_20',
         '2019_12_21',
         '2019_12_22',
         '2019_12_23',
         '2019_12_24',
         '2019_12_25',
         '2019_12_26',
         '2019_12_27',
         '2019_12_28',
         '2019_12_29',
         '2019_12_30',
         '2019_12_31',
         '2020_01_02',
         '2020_01_03',
         '2020_01_04',
         '2020_01_05',
         '2020_01_06',
         '2020_01_07',
         '2020_01_08']

log_dirs = []
for date in dates:
    ld = get_matching_log_dirs(date, root_dir=root_dir)
    for log_dir in ld:
        if 'x011' in log_dir or 'v011' in log_dir:
            continue
        else:
            log_dirs += [log_dir]
    #log_dirs += [*ld]
    
print(len(log_dirs))
log_dirs

In [ ]:
from gauge_inference_np import inference_plots
from runners.runner_np import create_dynamics, run_inference_np
from utils.file_io import timeit
import utils.file_io as io
from runners.runner_np import load_pkl
from config import NetWeights

#log_dir = os.path.abspath('../../gauge_logs/2020_01_07/L8_b64_lf2_v101_f32_0023/')
log_dir = log_dirs[-1]
#for log_dir in log_dirs:
    #if os.path.isfile(os.path.join(log_dir, 'weights.pkl')):
try:
    dynamics, lattice = create_dynamics(log_dir)
    
    params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
    net_weights = NetWeights(x_scale=1, x_translation=1, x_transformation=1,
                             v_scale=1, v_translation=1, v_transformation=1)
    rp_l2hmc = {
        'beta': 5.,
        'eps': dynamics.eps,
        'net_weights': net_weights,
        'run_steps': 5000,
    }

    rp_l2hmc, rd_l2hmc, ed_l2hmc = run_inference_np(log_dir, dynamics,
                                                    lattice, rp_l2hmc,
                                                    init='rand')
    #inference_plots(rd_l2hmc, ed_l2hmc, params, rp_l2hmc)
except:
    io.log(f'Unable to load from `weights.pkl` file in {log_dir}. Skipping!')

In [ ]:
from gauge_inference_np import inference_plots
inference_plots(rd_l2hmc, ed_l2hmc, params, rp_l2hmc)

In [ ]:
import arviz as az
import xarray as xr

In [ ]:
from lattice.lattice import u1_plaq_exact

def therm_arr(arr, therm_frac=0.25):
    num_steps = arr.shape[0]
    therm_steps = int(therm_frac * num_steps)
    arr = arr[therm_steps:, :]
    steps = np.arange(therm_steps, num_steps)
    return arr, steps
    
rd_dict = {}
for key, val in rd_l2hmc.items():
    if 'mask' in key:
        continue
        
    arr, steps = therm_arr(np.array(val))
    arr = arr.T
    
    if 'plaqs' in key:
        key = 'plaqs_diffs'
        arr = u1_plaq_exact(rp_l2hmc['beta']) - arr
        
    if 'charges' in key:
        arr = np.around(arr)
        
    rd_dict[key] = xr.DataArray(arr,
                                dims=['chain', 'draw'],
                                coords=[np.arange(arr.shape[0]), steps])

In [ ]:
ds_l2hmc = xr.Dataset(rd_dict)

In [ ]:
az.plot_forest(ds_l2hmc, kind='ridgeplot', var_names=['plaqs_diffs'], ridgeplot_alpha=0.4, ridgeplot_overlap=0.1, combined=False)

In [ ]:
az.plot_forest(ds_l2hmc, kind='ridgeplot', var_names=['charges'], combined=False)

In [ ]:
ds_l2hmc = xr.Dataset(rd_dict)

In [ ]:
sns.set_palette('bright')

In [ ]:
from gauge_inference_np import _get_title

In [ ]:
%matplotlib notebook

In [ ]:
%matplotlib inline

In [ ]:
plt.tight_layout()
axes = az.plot_trace(ds_l2hmc, combined=True, var_names=['plaqs_diffs', 'accept_prob', 'dx', 'charges'])
title_str = _get_title(params, rp_l2hmc)
#axes[0][0].set_title(title_str, fontsize='x-large')
fig = plt.gcf()
fig.suptitle(_get_title(params, rp_l2hmc), fontsize='22', y=1.05)

In [ ]:
plt.tight_layout()

In [ ]:
az.plot_forest(ds_l2hmc, v)

In [ ]:
steps.shape

In [ ]:
ds_dict = {}
for key, val in rd_dict.items():
    ds_dict[key] = xr.DataArray(val, coords=[np.arange(val.shape[0]), steps], dims=['chain', 'draw'])

In [ ]:
ds_l2hmc = xr.Dataset(ds_dict)

In [ ]:
az.plot_trace(ds_l2hmc, var_names=['plaqs_diffs', 'actions', 'accept_prob', 'dx', 'charges'], compact=True, combined=True)

In [ ]:
az.plot_trace(rd_dict['plaqs'])

In [ ]:
for key, val in ds_dict.items():
    az.plot_trace(ds_l2hmc)
    
    i

In [ ]:
%debug

In [ ]:
ds_dict.keys()

In [ ]:
az.plot_trace(rd_dict, var_names=['plaqs', 'accept_prob', 'actions', 'charges', 'dx'])

In [ ]:
ds_l2hmc = xr.Dataset(ds_dict)

In [ ]:
ds_l2hmc

In [ ]:
az.plot_trace(ds_dict)

In [ ]:
#foo = xr.DataArray(data, coords=[times, locs], dims=['time', 'space'])
plaqs_da = xr.DataArray(rd_dict['plaqs'])


In [ ]:
df_l2hmc = pd.DataFrame(rd_dict, )

In [ ]:
import arviz as az

In [ ]:
az.plot_trace(df_l2hmc, )

In [ ]:
for key, val in rd_l2hmc.items():
    
    
    print(f'{key}: {np.array(val).shape}')

In [ ]:
try:
    dynamics, lattice = create_dynamics(log_dir)
    net_weights = NetWeights(x_scale=0, x_translation=0, x_transformation=0,
                             v_scale=0, v_translation=0, v_transformation=0)
    rp_hmc = {
        'beta': 5.,
        'eps': dynamics.eps,
        'net_weights': net_weights,
        'run_steps': 500,
    }

    rp_hmc, rd_hmc, ed_hmc = run_inference_np(log_dir, dynamics,
                                              lattice, rp_hmc,
                                              init='rand')
    params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
    inference_plots(rd_hmc, ed_hmc, params, rp_hmc)
#else:
except:
    io.log(f'Unable to load from `weights.pkl` file in {log_dir}. Skipping!')
    #continue

In [ ]:
%debug

In [ ]:
run_params


In [ ]:
from runners.runner_np import load_pkl
params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
inference_plots(run_data, energy_data, params, run_params)

In [ ]:
def calc_dx(x1, x2):
    return np.mean(1. - np.cos(x1 - x2), axis=-1)

def _calc_energies(dynamics, x, v, beta):
    pe = dynamics.potential_energy(x, beta)
    ke = dynamics.kinetic_energy(v)
    h = dynamics.hamiltonian(x, v, beta)
    
    return pe, ke, h
    
def calc_energies(dynamics, x_init, outputs, beta):
    #x_init = outputs['x_init']
    x_prop = outputs['x_proposed']
    x_out = outputs['x_out']
    v_init = outputs['v_init']
    v_prop = outputs['v_proposed']
    v_out = outputs['v_out']
    
    pe_init, ke_init, h_init = _calc_energies(dynamics, x_init, v_init, beta)
    pe_prop, ke_prop, h_prop = _calc_energies(dynamics, x_prop, v_prop, beta)
    pe_out, ke_out, h_out = _calc_energies(dynamics, x_out, v_out, beta)
    
    outputs = { 
        'potential_init': pe_init,
        'kinetic_init': ke_init,
        'hamiltonian_init': h_init, 
        'potential_proposed': pe_prop,
        'kinetic_proposed': ke_prop,
        'hamiltonian_proposed': h_prop, 
        'potential_out': pe_out,
        'kinetic_out': ke_out,
        'hamiltonian_out': h_out,
    }
    
    return outputs

## Load information about saved model... 

By loading in the saved weights from the trained model, we can reconstruct the neural network using a pure   
numpy implementation which can then be used to run inference without having to reconstruct a tensorflow graph.

In [ ]:
def create_lattice(params):
    dim = params['dim']
    num_steps = params['num_steps']
    time_size = params['time_size']
    space_size = params['space_size']
    batch_size = params['batch_size']
    zero_masks = params['zero_masks']
    x_dim = (params['time_size']
             * params['space_size']
             * params['dim'])
    
    lattice = GaugeLattice(time_size=time_size,
                           space_size=space_size,
                           dim=dim, link_type='U1',
                           batch_size=batch_size)
    return lattice

In [ ]:
def get_eps(log_dir):
    run_dirs = get_run_dirs(log_dir)
    run_dir = run_dirs[0]
    rp_file = os.path.join(run_dir, 'run_params.pkl')
    with open(rp_file, 'rb') as f:
        run_params = pickle.load(f)
        
    lattice = create_lattice(params)
    potential_fn = lattice.calc_actions_np

    eps = run_params['eps']
    
    return eps

In [ ]:
def update_params(params, eps=None, num_steps=None, batch_size=None):
    if num_steps is None:
        num_steps = params['num_steps']
    else:
        params['num_steps'] = num_steps
        
    if batch_size is None:
        batch_size = params['batch_size']
    else:
        params['batch_size'] = batch_size
        
    if eps is None:
        eps = get_eps(log_dir)
    else:
        params['eps'] = eps
        
    return params

In [ ]:
def create_dynamics(log_dir, eps=None, num_steps=None, batch_size=None):
    params_file = os.path.join(log_dir, 'parameters.pkl')
    with open(params_file, 'rb') as f:
        params = pickle.load(f)
        
    params = update_params(params, eps,
                           num_steps,
                           batch_size)
    eps = params['eps']
    num_steps = params['num_steps']
    batch_size = params['batch_size']
    zero_masks = params['zero_masks']

    weights_file = os.path.join(log_dir, 'weights.pkl')
    with open(weights_file, 'rb') as f:
        weights = pickle.load(f)
        
    lattice = create_lattice(params)
    
    x_dim = lattice.x_dim
    potential_fn = lattice.calc_actions_np
    
    # Use numpy implementation of the action 
    # calculation for the potential function
    dynamics = DynamicsRunner(potential_fn, weights,
                              eps=eps, x_dim=x_dim,
                              num_steps=num_steps,
                              batch_size=batch_size,
                              zero_masks=zero_masks)
    return dynamics, lattice, params

## Re-construct `GaugeLattice` and build `DynamicsRunner`...

 - The `GaugeLattice` provides the definition of the `potential_function` and allows   
   us to easily calculate lattice observables from our inference samples.

 - The `DynamicsRunner` is the tensorflow-independent implementation of the `Dynamics`   
   engine used for training the L2HMC sampler.

In [ ]:
from config import NetWeights
from lattice.lattice import GaugeLattice
from plotters.plot_utils import get_run_dirs
from dynamics.dynamics_np import DynamicsRunner

log_dir = os.path.abspath('/home/foremans/DLHMC/l2hmc-qcd/gauge_logs/'
                          '2020_01_08/L8_b64_lf1_f32/')

dynamics, lattice = create_dynamics(log_dir) 

In [ ]:
#lattice = GaugeLattice(time_size=time_size,
#                       space_size=space_size,
#                       dim=dim, link_type='U1',
#                       batch_size=batch_size)
#
## Use numpy implementation of the action 
## calculation for the potential function
#potential_fn = lattice.calc_actions_np
#dynamics = DynamicsRunner(potential_fn, weights,
#                          eps=eps, x_dim=x_dim,
#                          num_steps=num_steps,
#                          batch_size=batch_size,
#                          zero_masks=zero_masks)
#
#beta = 5.
#net_weights = NetWeights(1, 1, 1, 1, 1, 1)
#samples_init = np.random.randn(params['batch_size'], x_dim)
#outputs = dynamics.apply_transition(samples_init, beta, net_weights,
#                                    model_type='GaugeModel')
#
#for key, val in outputs.items():
#    print(f'{key}: {val.shape}\n')

In [ ]:
import time
from lattice.lattice import u1_plaq_exact
import utils.file_io as io

def run_inference_np(dynamics, run_params):
    run_steps = run_params['run_steps']
    beta = run_params['beta']
    net_weights = run_params['net_weights']
    eps = run_params.get('eps', None)
    batch_size = run_params.get('batch_size', None)
    if eps is None:
        eps = dynamics.eps
    if eps != dynamics.eps:
        dynamics.eps = eps
    if batch_size is None:
        batch_size = dynamics.batch_size
    if batch_size != dynamics.batch_size:
        dynamics.batch_size = batch_size

    run_data = {
        'plaqs': [],
        'actions': [],
        'charges': [],
        'dxf': [],
        'dxb': [],
        'dx': [],
        'accept_prob': [],
        'px': [],
        'mask_f': [],
        'mask_b': [],
    }

    energy_data = {
        'potential_init': [],
        'kinetic_init': [],
        'hamiltonian_init': [],
        'potential_proposed': [],
        'kinetic_proposed': [],
        'hamiltonian_proposed': [],
        'potential_out': [],
        'kinetic_out': [],
        'hamiltonian_out': [],
    }

    h_strf = ("{:^13s}" + 7 * "{:^12s}").format(
        "STEP", "t/STEP", "% ACC", "∆x", "∆xf", "∆xb", "exp(∆H)", "∆ø"
    )

    data_strs = []

    samples = np.random.randn(batch_size, dynamics.x_dim)
    plaq_exact = u1_plaq_exact(beta)

    nw_str = ''.join((str(int(i)) for i in net_weights))
    beta_str = f'{beta}'.replace('.', '')
    eps_str = f'{eps:.3g}'.replace('.', '')
    run_str = (f'steps{run_steps}_beta{beta_str}'
               f'_eps{eps_str}_nw{nw_str}_rand')

    run_params['run_str'] = run_str

    runs_dir = os.path.join(log_dir, 'runs_np')
    if os.path.isdir(os.path.join(runs_dir, run_str)):
        io.log(f'Existing run found! Creating new run_dir...')
        run_str += '_1'

    run_dir = os.path.join(runs_dir, run_str)
    io.check_else_make_dir(run_dir)

    start_time = time.time()
    for step in range(run_steps):
        if step % 100 == 0:
            print(len(h_strf) * '-')
            print(h_strf)
            print(len(h_strf) * '-')
        t0 = time.time()
        samples_init = np.mod(samples, 2 * np.pi)
        output = dynamics.apply_transition(samples, beta, net_weights,
                                           model_type='GaugeModel')
        dt = time.time() - t0
        samples = output['x_out']
        samples = np.mod(samples, 2 * np.pi)
        obs = lattice.calc_observables_np(samples=samples)
        for key, val in obs.items():
            run_data[key].append(val)

        xf = np.mod(output['xf'], 2 * np.pi) * output['mask_f'][:, None]
        xf0 = samples_init * output['mask_f'][:, None]

        xb = np.mod(output['xb'], 2 * np.pi) * output['mask_b'][:, None]
        xb0 = samples_init * output['mask_b'][:, None]

        dxf = calc_dx(xf0, xf)
        dxb = calc_dx(xb0, xb)
        dx = calc_dx(samples_init, samples)

        run_data['dxf'].append(dxf)
        run_data['dxb'].append(dxb)
        run_data['dx'].append(dx)
        run_data['accept_prob'].append(output['accept_prob'])

        plaq_diff = plaq_exact - obs['plaqs']

        edata = calc_energies(dynamics, samples_init, outputs, beta)
        for key, val in edata.items():
            energy_data[key].append(val)

        exp_dh = np.exp(edata['hamiltonian_init'] - edata['hamiltonian_out'])

        px = output['accept_prob']
        run_data['px'].append(px)
        data_str = (f"{step:>6g}/{run_steps:<6g} "
                    f"{dt:^11.4g} "
                    f"{px.mean():^11.4g} "
                    f"{dx.mean():^11.4g} "
                    f"{dxf.mean():^11.4g} "
                    f"{dxb.mean():^11.4g} "
                    f"{exp_dh.mean():^11.4g} "
                    f"{plaq_diff.mean():^11.4g}")
        print(data_str)
        data_strs.append(data_str)

    print(len(h_strf) * '-')
    print(f'Time to complete: {time.time() - start_time:.4g}s')

    #io.save_params(run_params, run_dir, name='run_params')
    io.save_dict(run_params, run_dir, name='run_params')

    run_history_file = os.path.join(run_dir, 'run_history.txt')
    io.log(f'Writing run history to: {run_history_file}...')
    with open(run_history_file, 'w') as f:
        for s in data_strs:
            f.write(f'{s}\n')

    run_data_file = os.path.join(run_dir, 'run_data.pkl')
    io.log(f'Saving run_data to {run_data_file}...')
    with open(run_data_file, 'wb') as f:
        pickle.dump(run_data, f)

    energy_data_file = os.path.join(run_dir, 'energy_data.pkl')
    io.log(f'Saving energy_data to {energy_data_file}...')
    with open(energy_data_file, 'wb') as f:
        pickle.dump(energy_data, f)

    observables_dir = os.path.join(run_dir, 'observables')
    io.check_else_make_dir(observables_dir)
    for key, val in run_data.items():
        out_file = os.path.join(observables_dir, f'{key}.pkl')
        io.log(f'Saving {key} to {out_file}...')
        with open(out_file, 'wb') as f:
            pickle.dump(np.array(val), f)
            
    return run_params, run_data, energy_data

In [ ]:
run_params = {
    'run_steps': 10000,
    'beta': 5.,
    'net_weights': NetWeights(1, 1, 1, 1, 1, 1),
    'eps': dynamics.eps,
    'batch_size': dynamics.batch_size,
}
run_params_l2hmc, run_data_l2hmc, energy_data_l2hmc = run_inference_np(dynamics, run_params)
#run_params_l2hmc, run_data_l2hmc, energy_data_l2hmc = run_inference_np(run_steps=20000, beta=5., net_weights=nw_l2hmc)

In [ ]:
run_params = {
    'run_steps': 10000,
    'beta': 5.,
    'net_weights': NetWeights(0, 0, 0, 0, 0, 0),
    'eps': dynamics.eps,
    'batch_size': dynamics.batch_size,
}
run_params_hmc, run_data_hmc, energy_data_hmc = run_inference_np(dynamics, run_params)

In [ ]:
from gauge_inference

In [ ]:
rd_l2hmc = os.path.join(runs_dir, 'steps2000_beta50_eps00402_nw111111_rand_1')
obs_dir_l2hmc = os.path.join(rd_l2hmc, 'observables')
plaqs_file_l2hmc = os.path.join(obs_dir_l2hmc, 'plaqs.pkl')
with open(plaqs_file_l2hmc, 'rb') as f:
    plaqs_l2hmc = pickle.load(f)

In [ ]:
plaqs_l2hmc.shape

In [ ]:
fig, ax = plt.subplots()
plaq_exact = u1_plaq_exact(beta)
y1 = plaq_exact - plaqs_l2hmc[1000:, :].flatten()
y2 = plaq_exact - np.array(run_data['plaqs'])[1000:, :].flatten()
sns.kdeplot(y1, shade=True, ax=ax, label='nw: (1, 1, 1, 1, 1, 1)')
sns.kdeplot(y2, shade=True, ax=ax, label='nw: (0, 0, 0, 0, 0, 0)')
xlim = ax.get_xlim()
ax.set_xlim(xlim[0], 0.1)

In [ ]:
np.mean(plaq_exact - plaqs_l2hmc[1000:, :].flatten())

In [ ]:
np.mean(plaq_exact - np.array(run_data['plaqs'])[1000:, :].flatten())

In [ ]:
samples_dict['x_init'][-1]

In [ ]:
samples_dict['x_proposed'][-1]

In [ ]:
samples_dict['x_out'][-1]

In [ ]:
rd = os.path.join(log_dir ,'runs', 'steps20000_beta50_eps00402_x000_v000_random/')
od = os.path.join(rd, 'observables')
rdata = {}
files = [os.path.join(od, f'{i}') for i in os.listdir(od) if i.endswith('.pkl')]
for f in files:
    name = f.split('/')[-1].rstrip('.pkl')
    with open(f, 'rb') as ff:
        rdata[name] = pickle.load(ff)
         

rdata = {}
files = [os.path.join(observables_dir, f'{i}') for i in os.listdir(observables_dir) if i.endswith('.pkl')]
for f in files:
    name = f.split('/')[-1].rstrip('.pkl')
    with open(f, 'rb') as ff:
        rdata[name] = pickle.load(ff)

energy_data_file = os.path.join(rd, 'energy_data_tf.pkl')
with open(energy_data_file, 'rb') as f:
    energy_data = pickle.load(f)

for key, val in energy_data.items():
    print(f'{key}: {np.array(val).shape}\n')

for key, val in run_data.items():
    print(f'{key}: {np.array(val).shape}\n')

for key, val in rdata.items():
    print(f'{key}: {np.array(val).shape}\n')

In [ ]:
from lattice.lattice import u1_plaq_exact

run_data_hmc = {
    'dxf': [],
    'dxb': [],
    'accept_prob': [],
}
observables_hmc = {
    'plaqs': [],
    'actions': [],
    'charges': [],
}

beta = 5.
x_dim = time_size * space_size * dim
samples = np.random.randn(batch_size, x_dim)
net_weights = NetWeights(0, 0, 0, 0, 0, 0)

plaq_exact = u1_plaq_exact(beta)

run_steps = 5000
for step in range(run_steps):
    output = dynamics.apply_transition(samples, beta, net_weights,
                                       model_type='GaugeModel')
    samples = output['x_out']
    
    
    samples = np.mod(samples, 2 * np.pi)
    obs = lattice.calc_observables_np(samples=samples)
    for key, val in obs.items():
        observables_hmc[key].append(val)
        
    dxf = calc_dx(output['x_init'], output['xf'])
    dxb = calc_dx(output['x_init'], output['xb'])
    
    run_data_hmc['dxf'].append(dxf)
    run_data_hmc['dxb'].append(dxb)
    run_data_hmc['accept_prob'].append(output['accept_prob'])
    
    plaq_diff = plaq_exact - obs['plaqs']
    
    px = np.mean(output['accept_prob'])
    print(f'step: {step}/{run_steps},  '
          f'accept_prob: {px:.3g},  '
          f'dxf: {dxf.mean():.3g},  '
          f'dxb: {dxb.mean():.3g},  '
          f'plaq_diff: {plaq_diff.mean():.4g} +/- {plaq_diff.std():.4g}')

In [ ]:
for key, val in observables.items():
    observables[key] = np.array(val).flatten()
    
observables['accept_prob'] = np.array(run_data['accept_prob']).flatten()

observables_df = pd.DataFrame(observables)

In [ ]:
for key, val in observables_hmc.items():
    observables_hmc[key] = np.array(val).flatten()
    
observables['accept_prob'] = np.array(run_data_hmc['accept_prob']).flatten()
observables_hmc_df = pd.DataFrame(observables_hmc)

In [ ]:
from plotters.plot_utils import bootstrap
def get_stats(arr, axis=0):
    avg, err, arr_ = bootstrap(arr, n_boot=n_boot)
    return arr_.mean(axis=axis).flatten(), arr_.std(axis=axis).flatten()

In [ ]:
n_boot = 2000

observables_bs = {}
for key, val in observables.items():
    val = val.reshape(run_steps, -1)
    avg, err = get_stats(val, n_boot) 
    
    avg_key = f'{key}_avg'
    err_key = f'{key}_err'
    observables_bs[avg_key] = avg
    observables_bs[err_key] = err

In [ ]:
observables_df['plaqs_diffs'] = u1_plaq_exact(beta) - observables_df['plaqs']

In [ ]:
observables_hmc_df['plaqs_diffs'] = u1_plaq_exact(beta) - observables_hmc_df['plaqs']

In [ ]:
data=data[data.dx < 10]

In [ ]:
fig, ax = plt.subplots()
y1 = observables_df[observables_df.plaqs_diffs <= 0.2]['plaqs_diffs']
y2 = observables_hmc_df[observables_hmc_df.plaqs_diffs <= 0.2]['plaqs_diffs']
sns.kdeplot(y1, shade=True, ax=ax, label='nw: (1, 1, 1, 1, 1, 1)')
sns.kdeplot(y2, shade=True, ax=ax, label='nw: (0, 0, 0, 0, 0, 0)')
xlim = ax.get_xlim()
ax.set_xlim(xlim[0], 0.15)

In [ ]:
plaqs_diffs = u1_plaq_exact(beta) - np.array(observables['plaqs'])
sns.kdeplot(plaqs_diffs, shade=True, )

In [ ]:
%debug

In [ ]:
observables['plaqs'][0].shape

In [ ]:
plaq_sums = lattice.calc_plaq_sums(samples)
print(f'samples.shape: {samples.shape}\n')
print(f'plaq_sums.shape: {plaq_sums.shape}\n')

In [ ]:
outputs.keys()

In [ ]:
np.mean(outputs['accept_prob'])

In [ ]:
np.mean(1. - np.cos(outputs['x_init'] - outputs['x_out']))

In [ ]:
outputs['accept_prob']

In [ ]:
masks_file = os.path.abspath('/home/foremans/figures_2020_01_08/dynamics_mask.pkl')
with open(masks_file, 'rb') as f:
    masks = pickle.load(f)
    

In [ ]:
masks.keys()

In [ ]:
masks['masks']

In [ ]:
masks['masks_inv']

In [ ]:
masks['masks_inv'][0].shape